In [71]:
import bnlearn as bn
from pgmpy.factors.discrete import TabularCPD
import csv
import numpy as np
import sys
import os, contextlib
from pprint import pprint

In [72]:
N = 100

p_input = {
    'studies' : [[0.36], [0.64]],
    'loneliness' : [[0.85], [0.15]],
    'overeating' : [[0.85, 0.35, 0.7, 0.15],
                    [0.15, 0.65, 0.3, 0.85]],
    'depression' : [[0.8, 0.3], [0.2, 0.7]]
}

edges_input = {
    'studies' : 'overeating',
    'loneliness' : 'overeating',
    'overeating' : 'depression'
}

In [73]:
edges = [(edge, edges_input[edge]) for edge in edges_input]
unconditional_nodes = [node for node in edges_input if node not in edges_input.values()]
cpds = []
for node in unconditional_nodes:
    cpds.append(TabularCPD(variable=node, variable_card=2, values=p_input[node]))
for node in p_input:
    if node not in unconditional_nodes:
        evidence = [edge[0] for edge in edges if edge[1] == node]
        cpds.append(TabularCPD(variable=node, variable_card=2, values=p_input[node], evidence=evidence, evidence_card=[2]*len(evidence)))

DAG = bn.make_DAG(edges)
DAG = bn.make_DAG(DAG, CPD=cpds)

[bnlearn] >bayes DAG created.
[bnlearn] >No changes made to existing bayes DAG.
[bnlearn] >Add CPD: studies
[bnlearn] >Add CPD: loneliness
[bnlearn] >Add CPD: overeating
[bnlearn] >Add CPD: depression
[bnlearn] >Checking CPDs..
[bnlearn] >Check for DAG structure. Correct: True


In [ ]:

with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        with open('out.csv', 'w', newline = '') as outFile:
            writer = csv.DictWriter(outFile, fieldnames=p_input.keys())
            writer.writeheader()
            for n in range(N):
                row = {}
                for node in unconditional_nodes:
                    row[node] = np.random.binomial(1, p_input[node][1][0])
                for node in p_input:
                    if node not in unconditional_nodes:
                        evidence = {}
                        for edge in edges:
                            if edge[1] == node:
                                evidence[edge[0]] = row[edge[0]]
                        cond_p = bn.inference.fit(DAG, variables=[node], evidence=evidence).df['p'][1]
                        row[node] = np.random.binomial(1, cond_p)
                writer.writerow(row)


In [75]:
def check_output(out_filename):
    with open(out_filename, newline ='') as outFile:
        output = csv.DictReader(outFile)
        fieldnames = output.fieldnames
        p = {}
        dist_p = {}
        diff_p = {}
        for field_name in fieldnames:
            p[field_name] = 0
            dist_p[field_name] = bn.inference.fit(DAG, variables=[field_name], evidence={}).df['p'][1]
        for line in output:
            for field_name in fieldnames:
                p[field_name] += int(line[field_name])
        for key in p:
            p[key] = p[key] / N
        print('From data:')
        print(p)
        print('From distribition:')
        print(dist_p)
        for field_name in fieldnames:
            diff_p[field_name] = abs(p[field_name] - dist_p[field_name])
        print('Difference:')
        print(diff_p)
        return p

In [76]:
p = check_output('out.csv')

dist_p_depression_overeating = bn.inference.fit(DAG, variables=['overeating'], evidence={'depression' : 1}).df['p'][1]

p_depression_overeating = 0
with open('out.csv', newline ='') as outFile:
    output = csv.DictReader(outFile)
    for line in output:
        if int(line['depression']) == 1:
            p_depression_overeating += int(line['overeating'])
p_depression_overeating = p_depression_overeating/N/p['depression']


print("P(depression|overeating) from data:")
print(p_depression_overeating)
print("P(depression|overeating) from distribution:")
print(dist_p_depression_overeating)
print("P(depression|overeating) difference")
print(abs(p_depression_overeating - dist_p_depression_overeating))

[bnlearn] >Variable Elimination..


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


+----+-----------+------+
|    |   studies |    p |
+====+===========+======+
|  0 |         0 | 0.36 |
+----+-----------+------+
|  1 |         1 | 0.64 |
+----+-----------+------+
[bnlearn] >Variable Elimination..


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


+----+--------------+------+
|    |   loneliness |    p |
+====+==============+======+
|  0 |            0 | 0.85 |
+----+--------------+------+
|  1 |            1 | 0.15 |
+----+--------------+------+
[bnlearn] >Variable Elimination..


...

+----+--------------+--------+
|    |   overeating |      p |
+====+==============+========+
|  0 |            0 | 0.6742 |
+----+--------------+--------+
|  1 |            1 | 0.3258 |
+----+--------------+--------+
[bnlearn] >Variable Elimination..


Eliminating: loneliness: 100%|██████████| 3/3 [00:00<00:00, 303.42it/s]


+----+--------------+--------+
|    |   depression |      p |
+====+==============+========+
|  0 |            0 | 0.6371 |
+----+--------------+--------+
|  1 |            1 | 0.3629 |
+----+--------------+--------+
From data:
{'studies': 0.59, 'loneliness': 0.18, 'overeating': 0.37, 'depression': 0.4}
From distribition:
{'studies': 0.64, 'loneliness': 0.15, 'overeating': 0.3258, 'depression': 0.3629}
Difference:
{'studies': 0.050000000000000044, 'loneliness': 0.03, 'overeating': 0.04420000000000002, 'depression': 0.03710000000000002}
[bnlearn] >Variable Elimination..


Eliminating: loneliness: 100%|██████████| 2/2 [00:00<00:00, 182.17it/s]

+----+--------------+----------+
|    |   overeating |        p |
+====+==============+==========+
|  0 |            0 | 0.371562 |
+----+--------------+----------+
|  1 |            1 | 0.628438 |
+----+--------------+----------+
P(depression|overeating) from data:
0.7000000000000001
P(depression|overeating) from distribution:
0.6284375861118764
P(depression|overeating) difference
0.07156241388812368
